In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
%cd /gdrive/My Drive/FirstCompetition

dataset_base ='dataset'
train_data_dir = dataset_base +'/training'

/gdrive/My Drive/FirstCompetition


In [3]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
# cose per valutare l'algorithm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

tfk = tf.keras
tfkl = tfk.layers
print(tf.__version__)

2.7.0


In [4]:
img_height = 256
img_width = 256
seed = 333
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [7]:
labels = ['Apple',        # 0
          'Blueberry',    # 1
          'Cherry',       # 2
          'Corn',         # 3
          'Grape',        # 4
          'Orange',       # 5
          'Peach',        # 6
          'Pepper',       # 7
          'Potato',       # 8
          'Raspberry',    # 9
          'Soybean',      # 10
          'Squash',       # 11
          'Strawberry',   # 12
          'Tomato'        # 13
          ]

Let's define a function to produce train_generator and validation_generator given some hyper-parameter.

In [9]:
def hyper_generation (shift, zoom, channel, filling, rotation, brightness) :
    batch_size = 32;
    train_datagen = ImageDataGenerator(
        width_shift_range=shift, height_shift_range=shift,
        zoom_range=zoom, channel_shift_range=channel,
        horizontal_flip=True, vertical_flip=True,
        fill_mode=filling,
        rotation_range=rotation,
        brightness_range=brightness, #foto già illuminate, sopra 2 ha poco senso, con 0.4 sono già molto scure --> questi commenti sono ottenuti nel prossimo coding box
        validation_split=0.1) # set validation split

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        color_mode = 'rgb',
        batch_size=batch_size,
        class_mode='categorical',
        subset='training',  # set as traing data
        seed = seed)

    validation_generator = train_datagen.flow_from_directory(
        train_data_dir, # same directory as training data
        target_size=(img_height, img_width),
        color_mode = 'rgb',
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation',  # set as validation data
        seed = seed)
    return train_generator, validation_generator;

shift=0.3
zoom = 0.2
channel = 0.2
filling = 'reflect'
rotation = 90
brightness = (0.5,1.5)
train_generator, validation_generator = hyper_generation (shift, zoom, channel, filling, rotation, brightness)

Found 15972 images belonging to 14 classes.
Found 1766 images belonging to 14 classes.


We show some generated images to determine the range of parameters to work on.

In [10]:
# Create some augmentation examples

image = next(train_generator)[0][4:16]
figure, ax = plt.subplots(3,4, figsize = (45,45))
for i in range(12):
    ax[int(i/4)][i%4].imshow(np.uint8(image[i]))

Output hidden; open in https://colab.research.google.com to view.

In [11]:
def build_model(input_shape):

    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    conv1 = tfkl.Conv2D(
        filters=8,
        kernel_size=(7, 7),
        strides = (2, 2),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(input_layer)
    conv2 = tfkl.Conv2D(
        filters=16,
        kernel_size=(7, 7),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(conv1)
    pool1 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv2)

    conv3 = tfkl.Conv2D(
        filters=32,
        kernel_size=(5, 5),
        strides = (2, 2),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool1)
    conv4 = tfkl.Conv2D(
        filters=32,
        kernel_size=(5, 5),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(conv3)
    pool2 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv4)

    conv5 = tfkl.Conv2D(
        filters=64,
        kernel_size=(5, 5),
        strides = (2, 2),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool2)
    conv6 = tfkl.Conv2D(
        filters=64,
        kernel_size=(5, 5),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(conv5)
    pool3 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv6)

    flattening_layer = tfkl.Flatten(name='Flatten')(pool3)
    flattening_layer = tfkl.Dropout(0.3, seed=seed)(flattening_layer)
    classifier_layer = tfkl.Dense(units=128, name='Classifier', kernel_initializer=tfk.initializers.GlorotUniform(seed), activation='relu')(flattening_layer)
    classifier_layer = tfkl.Dropout(0.3, seed=seed)(classifier_layer)
    output_layer = tfkl.Dense(units=14, activation='softmax', kernel_initializer=tfk.initializers.GlorotUniform(seed), name='Output')(classifier_layer)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model

In [12]:
input_shape = (256, 256, 3)

model = build_model(input_shape)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 256, 256, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 128, 128, 8)       1184      
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 16)      6288      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 16)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 32)        12832     
                                                                 
 conv2d_3 (Conv2D)           (None, 32, 32, 32)        25632     
                                                             

In [13]:
# Utility function to create folders and callbacks for training
from datetime import datetime

def create_folders_and_callbacks(model_name):

  exps_dir = os.path.join('test_data_augmentation')
  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir)

  now = datetime.now().strftime('%b%d_%H-%M-%S')

  exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)
      
  callbacks = []

  # Model checkpoint
  # ----------------
  ckpt_dir = os.path.join(exp_dir, 'ckpts')
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)

  ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'), 
                                                     save_weights_only=False, # True to save only weights
                                                     save_best_only=False) # True to save only the best epoch 
  callbacks.append(ckpt_callback)

  # Visualize Learning on Tensorboard
  # ---------------------------------
  tb_dir = os.path.join(exp_dir, 'tb_logs')
  if not os.path.exists(tb_dir):
      os.makedirs(tb_dir)
      
  # By default shows losses and metrics for both training and validation
  tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir, 
                                               profile_batch=0,
                                               histogram_freq=1)  # if > 0 (epochs) shows weights histograms
  callbacks.append(tb_callback)

  # Early Stopping
  # --------------
  es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
  callbacks.append(es_callback)

  return callbacks

In [14]:
# create class weights
class_distr = []
for i in range(14):
  class_distr.append(sum(1 for x in train_generator.labels if x == i))
  
weights = []
total = train_generator.labels.size

for i in range(14):
  weights.append((i,(1/class_distr[i]) * (total/14.0)))

class_weights  = {cat:weight for (cat,weight) in weights}

Let's now create a function to run the training with different data generation parameters.

In [15]:
def train_model(n, shift, zoom, rotation, brightness):

  # function that trains the model on augmented images generated with different parameters
  #               saves the trained model on a folder
  #               and stores the maximum of the accuracy on the validation set
  # n represents the name of the model (folder name will be name = "test/Model_" + str(n)) 
    
    channel = 0.2
    filling = 'reflect'
    
    input_shape = (256, 256, 3)
    batch_size = 32

    model = build_model(input_shape)
    model.summary()
    
    tf.autograph.set_verbosity(0)
    epochs =6 # the number of epochs can be changed
    callbacks = create_folders_and_callbacks(model_name='First_iteration')
    
    train_generator, validation_generator = hyper_generation (shift, zoom, channel, filling, rotation, (brightness,1.5))

    
    # Train the model
    history = model.fit(
              x = train_generator,
              batch_size=batch_size,
              epochs = epochs,
              validation_data = validation_generator,
              class_weight=class_weights,
              callbacks = callbacks).history
    
    # Save best epoch model
    
    name = "test/Model_" + str(n)
    model.save(name)
    
    return max(history['accuracy'])


In [ ]:
lst = []
##use append to add items to the list.

n=0
for shift in np.arange(0.1,0.5,0.2):
    for zoom in np.arange(0.1,0.4,0.2):
        for rotation in np.arange(30,90,30):
            for brightness in np.arange(0.5,1.5,0.5):
                n=n+1;
                score = train_model(n, shift, zoom, rotation, brightness)
                lst.append({'shift':shift,'zoom':zoom,'rotation':rotation,'brightness':brightness, 'score':score})

lst

We have obtained a list with all accuracy scores and we have saved the respective trained models. We proceed to test these models on the challange platform and then save the scores obtained in the data_list (non abbiamo potuto farlo perchè non avevamo abbastanza tentativi di test sulla piattaforma, di seguito trovate data_list = lst solo a scopo di presentazione):

In [ ]:
data_list = [{'shift': 0.1,
  'zoom': 0.1,
  'rotation': 30,
  'brightness': 0.5,
  'score': 0.479546457529068},
 {'shift': 0.1,
  'zoom': 0.1,
  'rotation': 30,
  'brightness': 1.0,
  'score': 0.47666478157043457},
 {'shift': 0.1,
  'zoom': 0.1,
  'rotation': 60,
  'brightness': 0.5,
  'score': 0.413017600774765},
 {'shift': 0.1,
  'zoom': 0.1,
  'rotation': 60,
  'brightness': 1.0,
  'score': 0.44408944249153137},
 {'shift': 0.1,
  'zoom': 0.30000000000000004,
  'rotation': 30,
  'brightness': 0.5,
  'score': 0.3857670724391937},
 {'shift': 0.1,
  'zoom': 0.30000000000000004,
  'rotation': 30,
  'brightness': 1.0,
  'score': 0.4290546774864197},
 {'shift': 0.1,
  'zoom': 0.30000000000000004,
  'rotation': 60,
  'brightness': 0.5,
  'score': 0.38050490617752075},
 {'shift': 0.1,
  'zoom': 0.30000000000000004,
  'rotation': 60,
  'brightness': 1.0,
  'score': 0.3906533718109131},
 {'shift': 0.30000000000000004,
  'zoom': 0.1,
  'rotation': 30,
  'brightness': 0.5,
  'score': 0.3256906569004059},
 {'shift': 0.30000000000000004,
  'zoom': 0.1,
  'rotation': 30,
  'brightness': 1.0,
  'score': 0.249765083193779},
 {'shift': 0.30000000000000004,
  'zoom': 0.1,
  'rotation': 60,
  'brightness': 0.5,
  'score': 0.3299505114555359},
 {'shift': 0.30000000000000004,
  'zoom': 0.1,
  'rotation': 60,
  'brightness': 1.0,
  'score': 0.29643550515174866},
 {'shift': 0.30000000000000004,
  'zoom': 0.30000000000000004,
  'rotation': 30,
  'brightness': 0.5,
  'score': 0.31554219126701355},
 {'shift': 0.30000000000000004,
  'zoom': 0.30000000000000004,
  'rotation': 30,
  'brightness': 1.0,
  'score': 0.30526843667030334},
 {'shift': 0.30000000000000004,
  'zoom': 0.30000000000000004,
  'rotation': 60,
  'brightness': 0.5,
  'score': 0.2686838209629059},
 {'shift': 0.30000000000000004,
  'zoom': 0.30000000000000004,
  'rotation': 60,
  'brightness': 1.0,
  'score': 0.2938670814037323}]

In [ ]:
import pandas as pd
results = pd.DataFrame(data_list)
results

,shift,zoom,rotation,brightness,score
0,0.1,0.1,30,0.5,0.479546
1,0.1,0.1,30,1.0,0.476665
2,0.1,0.1,60,0.5,0.413018
3,0.1,0.1,60,1.0,0.444089
4,0.1,0.3,30,0.5,0.385767
5,0.1,0.3,30,1.0,0.429055
6,0.1,0.3,60,0.5,0.380505
7,0.1,0.3,60,1.0,0.390653
8,0.3,0.1,30,0.5,0.325691
9,0.3,0.1,30,1.0,0.249765
